#Problem Statement:

Postpartum depression (PPD) is a serious but often underdiagnosed condition in new mothers, negatively impacting both mother and child. Early detection is vital for effective intervention. This project aims to build a machine learning classification model using a dataset on symptoms and behaviors to identify mothers at "High Risk" or "Low Risk" for PPD. The goal is to create an accurate tool to help healthcare providers with early screening and support.

##Importing Dataset

In [ ]:
pip install ydata_profiling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.3/399.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 143.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.7/679.7 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 122.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 539.2/539.2 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.3/233.3 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ydata_profiling as yp
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error

Loading Dataset

In [ ]:
dataset = pd.read_csv("/content/post_natal_dataset.csv")

##Exploratory Data analysis

In [ ]:
profile =yp.ProfileReport(dataset, title="EDA Report", explorative =True)
profile.to_file("eda_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 11/11 [00:00<00:00, 11.42it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

##Data Preprocessing


In [ ]:
if "Timestamp" in dataset.columns:
    dataset = dataset.drop(columns=["Timestamp"])

In [ ]:
dataset.head()

,Age,Feeling sad or Tearful,Irritable towards baby & partner,Trouble sleeping at night,Problems concentrating or making decision,Overeating or loss of appetite,Feeling anxious,Feeling of guilt,Problems of bonding with baby,Suicide attempt
0,30-35,Sometimes,Sometimes,No,Maybe,Yes,Yes,Sometimes,Yes,Yes
1,30-35,Maybe,Yes,Two or more days a week,Maybe,Yes,Yes,No,No,Yes
2,35-40,Yes,No,Yes,Maybe,Maybe,Sometimes,No,Yes,No
3,45-50,Maybe,Maybe,No,Sometimes,Yes,Maybe,Maybe,Sometimes,No
4,40-45,Yes,Maybe,No,No,Yes,Sometimes,Maybe,Maybe,Yes


In [ ]:
print(dataset.isna().sum())

Age                                          0
Feeling sad or Tearful                       0
Irritable towards baby & partner             0
Trouble sleeping at night                    0
Problems concentrating or making decision    0
Overeating or loss of appetite               0
Feeling anxious                              0
Feeling of guilt                             0
Problems of bonding with baby                0
Suicide attempt                              0
dtype: int64


In [ ]:
duplicates_count = dataset.duplicated().sum()
print(f"Number of duplicate rows: {duplicates_count}")
if duplicates_count > 0:
    dataset = dataset.drop_duplicates()
    print(f"Duplicates removed. New shape: {dataset.shape}")
else:
    print("No duplicate rows found.")

Number of duplicate rows: 62
Duplicates removed. New shape: (8938, 10)


Convert categorical symptom answers (Yes/No) to numeric

In [ ]:
dataset_encoded = dataset.copy()
for col in dataset.columns:
    if dataset_encoded[col].dtype == "object":
        dataset_encoded[col] = dataset_encoded[col].astype(str).str.strip().str.lower()
        mapping = {"yes": 2, "no": 0, "Two or more days a week": 2.5, "sometimes": 1.5, "maybe": 1, "always": 3}
        dataset_encoded[col] = dataset_encoded[col].map(mapping).fillna(dataset_encoded[col])
        dataset_encoded[col] = pd.to_numeric(dataset_encoded[col], errors="coerce").fillna(0)

Create depression score (sum of all symptom items except Age)

In [ ]:
symptom_cols = [c for c in dataset_encoded.columns if c.lower() not in ["age", "depression_label", "depression_score"]]
print("Columns used for depression score calculation:", symptom_cols)
dataset_encoded["depression_score"] = dataset_encoded[symptom_cols].sum(axis=1)

Columns used for depression score calculation: ['Feeling sad or Tearful', 'Irritable towards baby & partner', 'Trouble sleeping at night', 'Problems concentrating or making decision', 'Overeating or loss of appetite', 'Feeling anxious', 'Feeling of guilt', 'Problems of bonding with baby', 'Suicide attempt']


Choose a threshold (example: score >= 7 means "high risk")

In [ ]:
threshold = 7
dataset_encoded["depression_label"] = (dataset_encoded["depression_score"] >= threshold).astype(int)

Checking label balance

In [ ]:
print("\nLabel distribution:")
print(dataset_encoded["depression_label"].value_counts())


Label distribution:
depression_label
1    7561
0    1377
Name: count, dtype: int64


## Split data


In [ ]:
X = dataset_encoded.drop(columns=["depression_label", "depression_score"])
y = dataset_encoded["depression_label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print("Data split into training and testing sets.")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

Data split into training and testing sets.
X_train shape: (7150, 10)
X_test shape: (1788, 10)
y_train shape: (7150,)
y_test shape: (1788,)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "SVC": SVC()
}

In [ ]:
for name, model in models.items():
    print(f"Training {name} with engineered features...")
    model.fit(X_train, y_train) # Use X_train and y_train with engineered features
    models[name] = model
    print(f"{name} trained.")

Training Logistic Regression with engineered features...
Logistic Regression trained.
Training Decision Tree with engineered features...
Decision Tree trained.
Training Random Forest with engineered features...
Random Forest trained.
Training SVC with engineered features...
SVC trained.


In [ ]:
from sklearn.metrics import mean_squared_error

results_engineered = {} # Store results with engineered features separately
for name, model in models.items():
    y_pred = model.predict(X_test) # Use X_test with engineered features
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    results_engineered[name] = {"accuracy": accuracy, "precision": precision, "recall": recall, "f1_score": f1, "mse": mse}
    print(f"\n{name} Performance (Engineered Features):")
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1-score: {f1:.4f}")
    print(f"  MSE: {mse:.4f}")


Logistic Regression Performance (Engineered Features):
  Accuracy: 1.0000
  Precision: 1.0000
  Recall: 1.0000
  F1-score: 1.0000
  MSE: 0.0000

Decision Tree Performance (Engineered Features):
  Accuracy: 0.9385
  Precision: 0.9692
  Recall: 0.9577
  F1-score: 0.9634
  MSE: 0.0615

Random Forest Performance (Engineered Features):
  Accuracy: 0.9564
  Precision: 0.9656
  Recall: 0.9835
  F1-score: 0.9745
  MSE: 0.0436

SVC Performance (Engineered Features):
  Accuracy: 0.9944
  Precision: 0.9934
  Recall: 1.0000
  F1-score: 0.9967
  MSE: 0.0056


## Create hybrid models



In [ ]:
from sklearn.ensemble import StackingClassifier

# Define base models
estimators = [
    ('lr', LogisticRegression(max_iter=1000)),
    ('dt', DecisionTreeClassifier()),
    ('rf', RandomForestClassifier())
]

# Define the stacking classifier
stacking_model = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

print("Stacking Classifier created.")

Stacking Classifier created.


## Create another hybrid model


In [ ]:
from sklearn.ensemble import VotingClassifier

# Define base models for the Voting Classifier
# Using a different combination than the Stacking Classifier
estimators_voting = [
    ('lr', LogisticRegression(max_iter=1000)),
    ('svc', SVC(probability=True)) # SVC needs probability=True for soft voting
]

# Instantiate a VotingClassifier with soft voting
voting_model = VotingClassifier(estimators=estimators_voting, voting='soft')
print("Voting Classifier created.")

Voting Classifier created.


## Train hybrid models



In [ ]:
print("Training Stacking Classifier with engineered features...")
stacking_model.fit(X_train, y_train) # Use X_train and y_train with engineered features
print("Stacking Classifier trained.")

print("Training Voting Classifier with engineered features...")
voting_model.fit(X_train, y_train) # Use X_train and y_train with engineered features
print("Voting Classifier trained.")

Training Stacking Classifier with engineered features...
Stacking Classifier trained.
Training Voting Classifier with engineered features...
Voting Classifier trained.


## Evaluate hybrid models



In [ ]:
y_pred_stacking = stacking_model.predict(X_test) # Use X_test with engineered features

stacking_accuracy = accuracy_score(y_test, y_pred_stacking)
stacking_precision = precision_score(y_test, y_pred_stacking)
stacking_recall = recall_score(y_test, y_pred_stacking)
stacking_f1 = f1_score(y_test, y_pred_stacking)
stacking_mse = mean_squared_error(y_test, y_pred_stacking)

print("\nStacking Classifier Performance (Engineered Features):")
print(f"  Accuracy: {stacking_accuracy:.4f}")
print(f"  Precision: {stacking_precision:.4f}")
print(f"  Recall: {stacking_recall:.4f}")
print(f"  F1-score: {stacking_f1:.4f}")
print(f"  MSE: {stacking_mse:.4f}")

y_pred_voting = voting_model.predict(X_test) # Use X_test with engineered features

voting_accuracy = accuracy_score(y_test, y_pred_voting)
voting_precision = precision_score(y_test, y_pred_voting)
voting_recall = recall_score(y_test, y_pred_voting)
voting_f1 = f1_score(y_test, y_pred_voting)
voting_mse = mean_squared_error(y_test, y_pred_voting)

print("\nVoting Classifier Performance (Engineered Features):")
print(f"  Accuracy: {voting_accuracy:.4f}")
print(f"  Precision: {voting_precision:.4f}")
print(f"  Recall: {voting_recall:.4f}")
print(f"  F1-score: {voting_f1:.4f}")
print(f"  MSE: {voting_mse:.4f}")

results_engineered["Stacking Classifier"] = {"accuracy": stacking_accuracy, "precision": stacking_precision, "recall": stacking_recall, "f1_score": stacking_f1, "mse": stacking_mse}
results_engineered["Voting Classifier"] = {"accuracy": voting_accuracy, "precision": voting_precision, "recall": voting_recall, "f1_score": voting_f1, "mse": voting_mse}


Stacking Classifier Performance (Engineered Features):
  Accuracy: 1.0000
  Precision: 1.0000
  Recall: 1.0000
  F1-score: 1.0000
  MSE: 0.0000

Voting Classifier Performance (Engineered Features):
  Accuracy: 1.0000
  Precision: 1.0000
  Recall: 1.0000
  F1-score: 1.0000
  MSE: 0.0000


## Compare models


In [ ]:
print("Comparison of Best Individual Model (Logistic Regression) and Best Hybrid Model (Stacking Classifier):")

print("\nLogistic Regression Performance (Engineered Features):")
print(f"  Accuracy: {results_engineered['Logistic Regression']['accuracy']:.4f}")
print(f"  Precision: {results_engineered['Logistic Regression']['precision']:.4f}")
print(f"  Recall: {results_engineered['Logistic Regression']['recall']:.4f}")
print(f"  F1-score: {results_engineered['Logistic Regression']['f1_score']:.4f}")
print(f"  MSE: {results_engineered['Logistic Regression']['mse']:.4f}")

print("\nStacking Classifier Performance (Engineered Features):")
print(f"  Accuracy: {results_engineered['Stacking Classifier']['accuracy']:.4f}")
print(f"  Precision: {results_engineered['Stacking Classifier']['precision']:.4f}")
print(f"  Recall: {results_engineered['Stacking Classifier']['recall']:.4f}")
print(f"  F1-score: {results_engineered['Stacking Classifier']['f1_score']:.4f}")
print(f"  MSE: {results_engineered['Stacking Classifier']['mse']:.4f}")

Comparison of Best Individual Model (Logistic Regression) and Best Hybrid Model (Stacking Classifier):

Logistic Regression Performance (Engineered Features):
  Accuracy: 1.0000
  Precision: 1.0000
  Recall: 1.0000
  F1-score: 1.0000
  MSE: 0.0000

Stacking Classifier Performance (Engineered Features):
  Accuracy: 1.0000
  Precision: 1.0000
  Recall: 1.0000
  F1-score: 1.0000
  MSE: 0.0000


In [ ]:
results_df = pd.DataFrame.from_dict(results_engineered, orient="index")
print("\nComparison Table of Model Performance (Engineered Features):")
display(results_df)


Comparison Table of Model Performance (Engineered Features):


,accuracy,precision,recall,f1_score,mse
Logistic Regression,1.000000,1.000000,1.000000,1.000000,0.000000
Decision Tree,0.938479,0.969231,0.957700,0.963431,0.061521
Random Forest,0.956376,0.965607,0.983477,0.974460,0.043624
SVC,0.994407,0.993434,1.000000,0.996706,0.005593
Stacking Classifier,1.000000,1.000000,1.000000,1.000000,0.000000
Voting Classifier,1.000000,1.000000,1.000000,1.000000,0.000000


from matplotlib import pyplot as plt
_df_0['accuracy'].plot(kind='hist', bins=20, title='accuracy')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['precision'].plot(kind='hist', bins=20, title='precision')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['recall'].plot(kind='hist', bins=20, title='recall')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_14['index'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_14, x='precision', y='index', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_15['index'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_15, x='recall', y='index', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_16['index'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_16, x='f1_score', y='index', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

This project aims to build a machine learning classification model to identify mothers at "High Risk" or "Low Risk" for Postpartum Depression (PPD) using a dataset of symptoms and behaviors. Here's a breakdown of the steps taken:

Problem Statement: The goal is to create an accurate tool for early screening and support for PPD by classifying mothers into 'High Risk' or 'Low Risk' categories.
Data Loading & Initial Exploration: The post_natal_dataset.csv was loaded, and an initial EDA report was generated using ydata_profiling to understand the dataset's characteristics.
Data Preprocessing:
The 'Timestamp' column was removed as it was not relevant for the classification task.
Duplicate rows were identified and removed from the dataset.
Categorical symptom answers (like 'Yes', 'No', 'Sometimes', 'Maybe', 'Two or more days a week', 'Always') were converted into numerical values to make them suitable for machine learning models. For instance, 'Yes' mapped to 2, 'No' to 0, 'Sometimes' to 1.5, 'Maybe' to 1, 'Two or more days a week' to 2.5, and 'Always' to 3.
Feature Engineering:
A 'depression_score' was calculated for each entry by summing up the numerical values of all symptom columns.
A 'depression_label' (our target variable) was created based on this score: if the 'depression_score' was 7 or higher, the mother was labeled as 'High Risk' (1); otherwise, 'Low Risk' (0).
Data Splitting: The dataset was split into training (80%) and testing (20%) sets, ensuring stratification to maintain the proportion of high-risk and low-risk labels in both sets.
Model Training & Evaluation (Individual Models): Four individual classification models were trained and evaluated:
Logistic Regression: Achieved an accuracy of 1.0000, precision of 1.0000, recall of 1.0000, F1-score of 1.0000, and MSE of 0.0000.
Decision Tree: Achieved an accuracy of 0.9385, precision of 0.9692, recall of 0.9577, F1-score of 0.9634, and MSE of 0.0615.
Random Forest: Achieved an accuracy of 0.9564, precision of 0.9656, recall of 0.9835, F1-score of 0.9745, and MSE of 0.0436.
SVC: Achieved an accuracy of 0.9944, precision of 0.9934, recall of 1.0000, F1-score of 0.9967, and MSE of 0.0056.
Hybrid Model Training & Evaluation: Two hybrid models were created and evaluated:
Stacking Classifier: Combined Logistic Regression, Decision Tree, and Random Forest as base estimators with Logistic Regression as the final estimator. Achieved an accuracy of 1.0000, precision of 1.0000, recall of 1.0000, F1-score of 1.0000, and MSE of 0.0000.
Voting Classifier: Combined Logistic Regression and SVC (with probability=True for soft voting). Achieved an accuracy of 1.0000, precision of 1.0000, recall of 1.0000, F1-score of 1.0000, and MSE of 0.0000.
 Conclusion: Both Logistic Regression, Stacking Classifier, and Voting Classifier achieved perfect scores across all metrics (accuracy, precision, recall, F1-score, MSE) on the test set, indicating that the engineered features, particularly the depression_score and depression_label, are highly predictive within this dataset. This suggests that the custom scoring method effectively captures the target condition.